### В этом блокноте мы рассмотрим методы оценки задач абстрактной суммаризации на простом примере. Мы изучим традиционные методы оценки, такие как ROUGE и BERTScore.

In [ ]:
# !pip install rouge --quiet
# !pip install bert_score --quiet

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
import re
import pandas as pd

# Python Implementation of the ROUGE Metric
from rouge import Rouge

# BERTScore leverages the pre-trained contextual embeddings from BERT and matches words in candidate and reference sentences by cosine similarity.
from bert_score import BERTScorer

# Example task
Для целей этого блокнота мы будем использовать пример саммари, приведенный ниже. Обратите внимание, что мы предоставляем два сгенерированных саммари для сравнения, а также эталонное резюме, написанное человеком, которое требуется для таких метрик оценки, как ROUGE и BERTScore.

Отрывок (фрагмент):

```OpenAI's mission is to ensure that artificial general intelligence (AGI) benefits all of humanity. OpenAI will build safe and beneficial AGI directly, but will also consider its mission fulfilled if its work aids others to achieve this outcome. OpenAI follows several key principles for this purpose. First, broadly distributed benefits - any influence over AGI's deployment will be used for the benefit of all, and to avoid harmful uses or undue concentration of power. Second, long-term safety - OpenAI is committed to doing the research to make AGI safe, and to promote the adoption of such research across the AI community. Third, technical leadership - OpenAI aims to be at the forefront of AI capabilities. Fourth, a cooperative orientation - OpenAI actively cooperates with other research and policy institutions, and seeks to create a global community working together to address AGI's global challenges.```

Саммари:

Референс Summary `/ref_summary` (сгенерированное человеком)	

```OpenAI aims to ensure artificial general intelligence (AGI) is used for everyone's benefit, avoiding harmful uses or undue power concentration. It is committed to researching AGI safety, promoting such studies among the AI community. OpenAI seeks to lead in AI capabilities and cooperates with global research and policy institutions to address AGI's challenges.```

Eval Summary 1 / eval_summary_1 (system generated)

```OpenAI aims to AGI benefits all humanity, avoiding harmful uses and power concentration. It pioneers research into safe and beneficial AGI and promotes adoption globally. OpenAI maintains technical leadership in AI while cooperating with global institutions to address AGI challenges. It seeks to lead a collaborative worldwide effort developing AGI for collective good.```

Eval Summary 2 / eval_summary_2 (system generated)

```OpenAI aims to ensure AGI is for everyone's use, totally avoiding harmful stuff or big power concentration. Committed to researching AGI's safe side, promoting these studies in AI folks. OpenAI wants to be top in AI things and works with worldwide research, policy groups to figure AGI's stuff.```

In [ ]:
excerpt = "OpenAI's mission is to ensure that artificial general intelligence (AGI) benefits all of humanity. OpenAI will build safe and beneficial AGI directly, but will also consider its mission fulfilled if its work aids others to achieve this outcome. OpenAI follows several key principles for this purpose. First, broadly distributed benefits - any influence over AGI's deployment will be used for the benefit of all, and to avoid harmful uses or undue concentration of power. Second, long-term safety - OpenAI is committed to doing the research to make AGI safe, and to promote the adoption of such research across the AI community. Third, technical leadership - OpenAI aims to be at the forefront of AI capabilities. Fourth, a cooperative orientation - OpenAI actively cooperates with other research and policy institutions, and seeks to create a global community working together to address AGI's global challenges."
ref_summary = "OpenAI aims to ensure artificial general intelligence (AGI) is used for everyone's benefit, avoiding harmful uses or undue power concentration. It is committed to researching AGI safety, promoting such studies among the AI community. OpenAI seeks to lead in AI capabilities and cooperates with global research and policy institutions to address AGI's challenges."
eval_summary_1 = "OpenAI aims to AGI benefits all humanity, avoiding harmful uses and power concentration. It pioneers research into safe and beneficial AGI and promotes adoption globally. OpenAI maintains technical leadership in AI while cooperating with global institutions to address AGI challenges. It seeks to lead a collaborative worldwide effort developing AGI for collective good."
eval_summary_2 = "OpenAI aims to ensure AGI is for everyone's use, totally avoiding harmful stuff or big power concentration. Committed to researching AGI's safe side, promoting these studies in AI folks. OpenAI wants to be top in AI things and works with worldwide research, policy groups to figure AGI's stuff."

## Evaluating using ROUGE

**ROUGE** (Recall-Oriented Understudy for Gisting Evaluation), в первую очередь оценивает совпадение слов между сгенерированным результатом и эталонным текстом. Это распространенная метрика для оценки задач автоматической суммаризации. Среди ее разновидностей есть `ROUGE-L`, которая позволяет определить, насколько хорошо система сохраняет суть оригинального саммари - это самое длинное непрерывное совпадение между сгенерированным системой и эталонным саммари.

In [ ]:
# function to calculate the Rouge score
def get_rouge_scores(text1, text2):
    rouge = Rouge()
    return rouge.get_scores(text1, text2)


rouge_scores_out = []

# Calculate the ROUGE scores for both summaries using reference
eval_1_rouge = get_rouge_scores(eval_summary_1, ref_summary)
eval_2_rouge = get_rouge_scores(eval_summary_2, ref_summary)

for metric in ["rouge-1", "rouge-2", "rouge-l"]:
    for label in ["F-Score"]:
        eval_1_score = eval_1_rouge[0][metric][label[0].lower()]
        eval_2_score = eval_2_rouge[0][metric][label[0].lower()]

        row = {
            "Metric": f"{metric} ({label})",
            "Summary 1": eval_1_score,
            "Summary 2": eval_2_score,
        }
        rouge_scores_out.append(row)


def highlight_max(s):
    is_max = s == s.max()
    return [
        "background-color: lightgreen" if v else "background-color: white"
        for v in is_max
    ]


rouge_scores_out = (
    pd.DataFrame(rouge_scores_out)
    .set_index("Metric")
    .style.apply(highlight_max, axis=1)
)

rouge_scores_out

В таблице показаны оценки ROUGE для оценки двух различных саммари по сравнению с эталонным текстом. 
В случае с rouge-1 саммари 2 превосходит саммари 1, что свидетельствует о лучшем совпадении отдельных слов, а в случае с rouge-l саммари 2 имеет более высокий балл, что означает более тесное совпадение по самым длинным общим подпоследовательностям и, таким образом, потенциально лучшее общее обобщение, передающее основное содержание и порядок оригинального текста. 

Поскольку в саммари 2 много слов и коротких фраз, взятых непосредственно из отрывка, его совпадение с эталонным резюме, скорее всего, будет выше, что приведет к более высоким оценкам ROUGE.

Хотя ROUGE и аналогичные метрики, такие как BLEU и METEOR, предлагают количественные показатели, они часто не отражают истинную суть хорошо сгенерированного саммари. Кроме того, они хуже коррелируют с человеческими оценками. Учитывая развитие LLM, которые умеют создавать беглые и связные саммари, традиционные метрики, такие как ROUGE, могут невольно штрафовать эти модели. Это особенно верно, если саммари сформулированы по-разному, но при этом точно отражают основную информацию.

## Evaluating using BERTScore

ROUGE опирается на точное присутствие слов в предсказанных и эталонных текстах, не умея интерпретировать лежащую в их основе семантику. Именно здесь на помощь приходит BERTScore, который использует контекстуальные эмбеддинги из модели BERT, стремясь оценить сходство между предсказанным и эталонным предложением в контексте текста, созданного машиной. Сравнивая эмбеддинги из обоих предложений, BERTScore улавливает семантическое сходство, которое может быть упущено традиционными метриками, основанными на n-граммах.

In [ ]:
# Instantiate the BERTScorer object for English language
scorer = BERTScorer(lang="en")

# Calculate BERTScore for the summary 1 against the excerpt
# P1, R1, F1_1 represent Precision, Recall, and F1 Score respectively
P1, R1, F1_1 = scorer.score([eval_summary_1], [ref_summary])

# Calculate BERTScore for summary 2 against the excerpt
# P2, R2, F2_2 represent Precision, Recall, and F1 Score respectively
P2, R2, F2_2 = scorer.score([eval_summary_2], [ref_summary])

print("Summary 1 F1 Score:", F1_1.tolist()[0])
print("Summary 2 F1 Score:", F2_2.tolist()[0])

Близкие показатели F1 между саммари указывают на то, что они могут одинаково эффективно отражать ключевую информацию. Однако эту небольшую разницу следует интерпретировать с осторожностью. Поскольку BERTScore может не полностью улавливать тонкости и высокоуровневые понятия, которые может понять человек, опирающийся на эту метрику, может привести к неправильной интерпретации фактического качества и нюансов саммари. Интегрированный подход, сочетающий BERTScore с человеческим суждением и другими метриками, может предложить более надежную оценку.